# Perform initial setup

In [1]:
from theano.sandbox import cuda
%matplotlib inline
import utils
reload(utils)
from utils import *
from __future__ import division, print_function
path = ""
batch_size = 64
validation_size = 0.2

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [3]:
%pwd
%cd ..

/home/ubuntu


# Create a 20% validation set

Preserve the class imbalances in the training set

In [6]:
%cd "train"
species = glob("*")
for d in species:
    os.mkdir('../valid/'+d)
    images = glob(d+"/*")
    shuf = np.random.permutation(images)
    sample_size = int(round(len(images) * validation_size , 0))
    for i in range(sample_size): os.rename(shuf[i], '../valid/' + shuf[i])
%cd ..
%pwd

/home/ubuntu/train
/home/ubuntu


u'/home/ubuntu'

Read labels

In [7]:
(val_classes, trn_classes, val_labels , trn_labels, val_filenames, trn_filenames, test_filenames) = get_classes(path)

Found 3022 images belonging to 8 classes.
Found 755 images belonging to 8 classes.
Found 1000 images belonging to 1 classes.


In [32]:
raw_filenames = [f.split('/')[-1] for f in trn_filenames]
raw_test_filenames = [f.split('/')[-1] for f in test_filenames]
raw_val_filenames = [f.split('/')[-1] for f in val_filenames]

# Build CNN Model

In [11]:
vgg640 = Vgg16BN((360,640)).model
vgg640.pop()
#vgg640.input_shape()
#vgg640.output_shape()
vgg640.compile(Adam(), 'categorical_crossentropy' , metrics = ['accuracy'])

# Get data in 360 X 640 format and augment it

Introduce random variations in the training data including rotation, width shift, zoom etc

In [12]:
gen = image.ImageDataGenerator(rotation_range=10, width_shift_range=0.1, height_shift_range=0.1, 
                               shear_range=0.15, zoom_range = 0.1 , channel_shift_range = 10. , horizontal_flip = True)

In [13]:
batches = get_batches(path+'train', gen,shuffle=False, batch_size=1, class_mode=None, target_size=(360,640))
trn = np.concatenate([batches.next() for i in range(batches.nb_sample)])

batches = get_batches(path+'valid', shuffle=False, batch_size=1, class_mode=None, target_size=(360,640))
val = np.concatenate([batches.next() for i in range(batches.nb_sample)])

Found 3022 images belonging to 8 classes.
Found 755 images belonging to 8 classes.


# Pre compute features

In [14]:
trn_features = vgg640.predict(trn, batch_size = 32, verbose =1)
val_features = vgg640.predict(val, batch_size = 32, verbose =1)
del trn, val

755/755 [==============================] - 95s    


Repeat for test data

In [15]:
batches = get_batches(path+'test', shuffle=False, batch_size=1, class_mode=None, target_size=(360,640))
test = np.concatenate([batches.next() for i in range(batches.nb_sample)])
test_features = vgg640.predict(test, batch_size = 32, verbose =1)
del test

Found 1000 images belonging to 1 classes.
1000/1000 [==============================] - 126s   


Save a copy or load it from disk, if available 

################################################    RESUME FROM HERE   ###################################################

In [16]:
FIRST_TIME = True
if(FIRST_TIME):
    save_array(path+'val_features.dat', val_features)
    save_array(path+'trn_features.dat', trn_features)
    save_array(path+'test_features.dat', test_features)
else:
    val_features = load_array(path+'val_features.dat')
    trn_features = load_array(path+'trn_features.dat')
    test_features = load_array(path+'test_features.dat')

# Fit Model

Remove the final dense layer making this is a fully convolutional net

In [48]:
conv_layers,_ = split_at(vgg640,Convolution2D)
nf=128
p = 0.25

Add four convolution layers

In [49]:
def get_final_layers():
    return [
        BatchNormalization(axis=1, input_shape=conv_layers[-1].output_shape[1:]),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
        MaxPooling2D((1,2)),
        Convolution2D(8,3,3, border_mode='same'),
        Dropout(p),
        GlobalAveragePooling2D(),
        Activation('softmax')
    ]
myModel = Sequential(get_final_layers())
myModel.summary()

th
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_13 (BatchNorma(None, 512, 22, 40)   1024        batchnormalization_input_4[0][0] 
____________________________________________________________________________________________________
convolution2d_26 (Convolution2D) (None, 128, 22, 40)   589952      batchnormalization_13[0][0]      
____________________________________________________________________________________________________
batchnormalization_14 (BatchNorma(None, 128, 22, 40)   256         convolution2d_26[0][0]           
____________________________________________________________________________________________________
maxpooling2d_15 (MaxPooling2D)   (None, 128, 11, 20)   0           batchnormalization_14[0][0]      
________________________________________________________________________________________

Train a couple of epochs with high learning rate

In [50]:
myModel.compile(Adam(lr=0.001),loss='categorical_crossentropy' , metrics = ['accuracy'])
myModel.fit(trn_features, trn_labels, batch_size = batch_size, nb_epoch = 3 , validation_data = (val_features, val_labels))

Train on 3022 samples, validate on 755 samples
Epoch 1/3
3022/3022 [==============================] - 14s - loss: 1.0322 - acc: 0.6400 - val_loss: 2.5454 - val_acc: 0.5960
Epoch 2/3
3022/3022 [==============================] - 14s - loss: 0.3038 - acc: 0.9120 - val_loss: 1.2869 - val_acc: 0.6543
Epoch 3/3
3022/3022 [==============================] - 14s - loss: 0.1247 - acc: 0.9619 - val_loss: 0.5624 - val_acc: 0.8305


Train another 6 epochs with low learning rate

In [51]:
myModel.optimizer.lr=1e-4
myModel.fit(trn_features, trn_labels, batch_size = batch_size, nb_epoch = 6 , validation_data = (val_features, val_labels))
myModel.save_weights(path+'myModel.h5')
myModel.evaluate(val_features,val_labels)

Train on 3022 samples, validate on 755 samples
Epoch 1/6
3022/3022 [==============================] - 14s - loss: 0.0690 - acc: 0.9831 - val_loss: 0.4402 - val_acc: 0.8543
Epoch 2/6
3022/3022 [==============================] - 14s - loss: 0.0475 - acc: 0.9844 - val_loss: 0.4231 - val_acc: 0.9099
Epoch 3/6
3022/3022 [==============================] - 14s - loss: 0.0693 - acc: 0.9815 - val_loss: 0.6725 - val_acc: 0.8477
Epoch 4/6
3022/3022 [==============================] - 14s - loss: 0.0535 - acc: 0.9851 - val_loss: 0.6150 - val_acc: 0.8384
Epoch 5/6
3022/3022 [==============================] - 14s - loss: 0.0279 - acc: 0.9921 - val_loss: 0.4554 - val_acc: 0.8954
Epoch 6/6
755/755 [==============================] - 1s     


[0.39876778318195155, 0.91390728532083776]

# Make a Submission

In [52]:
def do_clip(arr, mx): return np.clip(arr, (1-mx)/7, mx)
predictions = myModel.predict(test_features, batch_size = batch_size)
predictions_clipped = do_clip(predictions, 0.72)
classes = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']
submission = pd.DataFrame(predictions_clipped, columns=classes)
submission.insert(0, 'image', raw_test_filenames)
submission.head()
subm_name = path+'submission.csv'
submission.to_csv(subm_name, index=False)